In [1]:
from specpy import *
from Util.coordinates import *
from Util.datastructures import *
from Util.tile_util import *
from Util.imspector_util import *
from Util.dot_detection.Fiji_coordinate_detection import *
from Util.coordinate_util import *
from Util.dot_detection.Fiji_dummy import *
from Util.dot_detection.Fiji_coordinate_detection import read_coords
import pprint
from Util import datastructures, imspector_util
import time
from matplotlib import pyplot as plt
from skimage.feature import blob_dog, blob_log, blob_doh
from scipy import ndimage, spatial
%matplotlib inline

In [2]:
def detect_blobs(img, sigmas, threshold):
    img = skimage.util.img_as_float(img)
    img = img / np.max(img)
    logimg = -ndimage.gaussian_laplace(img, sigmas)
    peaks = skimage.feature.peak_local_max(logimg, threshold_abs=threshold)
    return peaks    

In [3]:
def pair_finder(ms, pix_sig=3, threshold=0.01):

    stack1 = ms.stack(0).data()[0,:,:,:]
    stack2 = ms.stack(1).data()[0,:,:,:]

    sig = pix_sig / np.sqrt(2)
    dets1 = detect_blobs(stack1, [sig, sig, sig], threshold)    
    dets1=list(dets1)
    dets2 = detect_blobs(stack2, [sig, sig, sig], threshold)    
    dets2=list(dets2)

    if (len(dets1) == 0 or len(dets2) == 0):
        return []
    
    kd1 = spatial.KDTree(dets1)
    kd2 = spatial.KDTree(dets2)

    cleanup_kdtree(stack1, kd1, dets1, 3)
    cleanup_kdtree(stack2, kd2, dets2, 3)

    res = []
    for p in find_pairs(kd2, dets1, 5):
        res.append(list(p))

    return res

In [4]:
def cleanup_kdtree(img, kdt, dets, dist):
    notdone = True
    while notdone:
        notdone = False
        for i in range(len(dets)):
            nn = kdt.query(dets[i],k=2, distance_upper_bound=dist)
            if (not np.isinf(nn[0][1])) and nn[0][1] > 0:
                ineighbor = nn[1][1]
                if img[tuple(dets[i])] < img[tuple(dets[ineighbor])]:
                    dets.pop(i)
                else:
                    dets.pop(ineighbor)
                notdone = True
                    
                kdt = spatial.KDTree(dets)
                break
        
def find_pairs(kdt2, dets1, dist):
    res = []
    for d in dets1:
        nn = kdt2.query(d, distance_upper_bound=dist)
        if not np.isinf(nn[0]):
            res.append(d)
    return res

In [8]:
im = Imspector()
ms = im.active_measurement()
pair_finder(ms)
im.close(ms)

In [9]:
im = Imspector()

see = Settings()
see.load_from_file('C:/Users//RESOLFT/Desktop/ov.json')
coords = generate_grid_oop([0,0],[2e-3, 2e-3], [5e-5, 5e-5])
for c in coords:
    c.set_fov_len([5e-05, 5e-05, 1.2e-5])
    
    
nm = datastructures.NameManagement('C:/Users//RESOLFT/Desktop/AUTOMATION/')
nm.add_counter('field')
nm.add_counter('sted')

for c in coords:
    see.set_to_coordinates(c)
    
    name = nm.get_next_image_name('field')
    acquire_measurement_dummy(im, see)
    ms = im.active_measurement()
    
    sites = pair_finder(ms)
    
    co = c.copy()
    
    corner = middle2corner(c.get_all_in_one_offset(), c.get_fov_len())
    pixelsd = get_pixel_dimensions(ms)
    actual_coords = corner2spot(corner, sites, pixelsd)
    
    ms.save_as(name)
    im.close(ms)
    
    print(sites)
    
    see2 = Settings()
    see2.load_from_file('C:/Users//RESOLFT/Desktop/det.json')
    
    nm.reset_counter('sted')
    
    for i in sites:
        co.set_fov_len([3e-06, 3e-06, 1e-06])
        co.set_offset_coords(i)
        see2.set_to_coordinates(co)
        
        name = nm.get_next_image_name('sted')
        acquire_measurement_dummy(im, see2)
        
        ms = im.active_measurement()
        ms.save_as(name)
        im.close(ms)

[[3, 203, 302]]


KeyboardInterrupt: 

In [20]:
stack1 = ms.stack(0).data()[0,:,:,:]
stack2 = ms.stack(1).data()[0,:,:,:]

sig = 3 / np.sqrt(2)
dets1 = detect_blobs(stack1, [sig, sig, sig], 0.002)    
dets1=list(dets1)
dets2 = detect_blobs(stack2, [sig, sig, sig], 0.002)    
dets2=list(dets2)


In [21]:
print(dets2)

[array([  4, 181, 191], dtype=int64), array([  8,  87, 102], dtype=int64), array([13, 65, 94], dtype=int64), array([ 17, 200, 174], dtype=int64), array([ 22, 151,  90], dtype=int64), array([ 25, 161, 166], dtype=int64)]
